In [ ]:
# --- Install deps ---
!pip -q install fastapi uvicorn pyngrok nest_asyncio

import time, threading
import nest_asyncio
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn

# Allow nested event loops (safe in notebooks)
nest_asyncio.apply()

# ====== CONFIG (optional) ======
AUTHTOKEN = "" # FILL HERE

if AUTHTOKEN:
    ngrok.set_auth_token(AUTHTOKEN)

# ====== FastAPI app ======
from typing import List
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI(title="Colab FastAPI Example — CRUD 2")

class Item(BaseModel):
    id: int
    name: str
    price: float
    in_stock: bool = True

# fake in-memory "database"
db: List[Item] = []

@app.get("/items")
def get_items():
    return db

@app.post("/items")
def add_item(item: Item):
    db.append(item)
    return {"message": "Item added!", "item": item}

@app.delete("/items/{item_id}")
def delete_item(item_id: int):
    for i, item in enumerate(db):
        if item.id == item_id:
            deleted = db.pop(i)
            return {"message": "Item deleted!", "deleted": deleted}
    raise HTTPException(status_code=404, detail="Item not found")


# ====== Start Uvicorn in background thread ======
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Tiny wait so server boots
time.sleep(1)

# ====== Create public tunnel ======
tunnel = ngrok.connect(8000, bind_tls=True)
public_url = tunnel.public_url
print("🚀 Public URL:", public_url)
print("📚 OpenAPI docs:", public_url + "/docs")
print("🔎 Redoc:", public_url + "/redoc")


INFO:     Started server process [285]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Public URL: https://e64a6e3d7b4b.ngrok-free.app
📚 OpenAPI docs: https://e64a6e3d7b4b.ngrok-free.app/docs
🔎 Redoc: https://e64a6e3d7b4b.ngrok-free.app/redoc


In [ ]:
import requests

BASE_URL = public_url

# --- Add new items (POST /items) ---
print("Adding items...")
r1 = requests.post(f"{BASE_URL}/items", json={"id": 1, "name": "Rose", "price": 10.5})
r2 = requests.post(f"{BASE_URL}/items", json={"id": 2, "name": "Tulip", "price": 8.0})
r3 = requests.post(f"{BASE_URL}/items", json={"id": 3, "name": "Lily", "price": 12.0})
print(r1.json())
print(r2.json())
print(r3.json())

# --- Show all items (GET /items) ---
print("\nCurrent list:")
print(requests.get(f"{BASE_URL}/items").json())

# --- Delete one item (DELETE /items/{item_id}) ---
print("\nDeleting item with id=2")
delete_response = requests.delete(f"{BASE_URL}/items/2")
print(delete_response.json())

# --- Check updated list ---
print("\nUpdated list:")
print(requests.get(f"{BASE_URL}/items").json())


Adding items...
INFO:     35.185.21.184:0 - "POST /items HTTP/1.1" 200 OK
INFO:     35.185.21.184:0 - "POST /items HTTP/1.1" 200 OK
INFO:     35.185.21.184:0 - "POST /items HTTP/1.1" 200 OK
{'message': 'Item added!', 'item': {'id': 1, 'name': 'Rose', 'price': 10.5, 'in_stock': True}}
{'message': 'Item added!', 'item': {'id': 2, 'name': 'Tulip', 'price': 8.0, 'in_stock': True}}
{'message': 'Item added!', 'item': {'id': 3, 'name': 'Lily', 'price': 12.0, 'in_stock': True}}

Current list:
INFO:     35.185.21.184:0 - "GET /items HTTP/1.1" 200 OK
[{'id': 1, 'name': 'Rose', 'price': 10.5, 'in_stock': True}, {'id': 2, 'name': 'Tulip', 'price': 8.0, 'in_stock': True}, {'id': 3, 'name': 'Lily', 'price': 12.0, 'in_stock': True}]

Deleting item with id=2
INFO:     35.185.21.184:0 - "DELETE /items/2 HTTP/1.1" 200 OK
{'message': 'Item deleted!', 'deleted': {'id': 2, 'name': 'Tulip', 'price': 8.0, 'in_stock': True}}

Updated list:
INFO:     35.185.21.184:0 - "GET /items HTTP/1.1" 200 OK
[{'id': 1, 'n

In [ ]:
a = 1